In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import t
import matplotlib.pyplot as plt

**(d) (30 puntos)**

En el mercado, el **VaR** y el **ES** son las medidas populares para medir el riesgo de una cartera o un activo, sin embargo, es común ver que usan _rolling windows_, i.e. fijar una ventana de $X$ días y con esos datos calcular el $VaR$ o $ES$ del día $X + 1$, ejemplo: con una ventana de 252 retornos $(r_1, r_2, ..., r_{252})$ calculas el $VaR_\alpha$ asociado al retorno $r_{253}$, ahora el $VaR_\alpha$ asociado al retorno $r_{254}$ viene del conjunto de datos $(r_2, r_3, ..., r_{253})$, etc.



Con esto en mente, **en una sola gráfica muestra las ganancias y pérdidas además del $VaR$ y el $ES$ con $\alpha = 0.95$ y $0.99$ con una _rolling window_ de 252 retornos** (debe de ser una serie de tiempo) y **sobre todo recuerda que el $VaR_\alpha^{t}$ es calculado con los retornos $r_1, r_2, ..., r_{252}$ y busca predecir el retorno $r_{253}$, y el $VaR_\alpha^{t+1}$ calculado con los retornos $r_2, r_3, ..., r_{253}$ busca predecir el retorno $r_{254}$, etc.**



**Asegúrate que tu código realice esta gráfica además de ponerla en el Streamlit.**  
La estimación del $VaR$ y $ES$ debe de ser **histórico y paramétrico** (puedes asumir una distribución normal por practicidad), recuerda el ejemplo que se vio en clase.

In [2]:
# ------------------------------------------
# 1. PARÁMETROS DE CONFIGURACIÓN
# ------------------------------------------

ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2025-03-28'
window = 252
alphas = [0.95, 0.99]

# ------------------------------------------
# 2. DESCARGA DE DATOS Y CÁLCULO DE RETORNOS
# ------------------------------------------

df = yf.download(ticker, start=start_date, end=end_date)
df['returns'] = df['Close'].pct_change()
df = df.dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [3]:
# ------------------------------------------
# 3. FUNCIÓN PARA CÁLCULO PARAMÉTRICO
# ------------------------------------------

"""
def parametric_var_es(returns, alpha):


    #Calcula el VaR y el ES asumiendo distribución normal.

    mu = returns.mean()
    sigma = returns.std()
    var = norm.ppf(1 - alpha, mu, sigma)
    es = returns[returns <= var].mean()
    return var, es

"""

def parametric_var_es_tstudent(returns, alpha, nu=5):

    """
    Calcula el VaR y el ES asumiendo una t-Student con grados de libertad fijos (nu).
    Estimadores insesgados para la media y varianza.

    """
    mu = np.mean(returns)
    s2 = np.var(returns, ddof=1)  #varianza muestral insesgada
    sigma2 = ((nu - 2) / nu) * s2
    sigma = np.sqrt(sigma2)

    var = t.ppf(1 - alpha, df=nu, loc=mu, scale=sigma)
    es = t.expect(lambda x: x, args=(nu,), loc=mu, scale=sigma, lb=-np.inf, ub=var)
    return var, es

def historical_es(returns, alpha):

    """
    Calcula el Expected Shortfall (ES) histórico.

    """

    var_threshold = returns.quantile(1 - alpha)
    return returns[returns <= var_threshold].mean()

# ------------------------------------------
# 4. ROLLING WINDOW PARA VAR Y ES
# ------------------------------------------

#Inicializamos el DataFrame para guardar resultados
results = pd.DataFrame(index=df.index)
results['returns'] = df['returns']

for alpha in alphas:
    #Columnas para guardar los resultados
    var_param_col = f'VaR_param_{int(alpha*100)}'
    es_param_col = f'ES_param_{int(alpha*100)}'
    var_hist_col = f'VaR_hist_{int(alpha*100)}'
    es_hist_col = f'ES_hist_{int(alpha*100)}'

        #VaR y ES paramétrico con t-Student
    results[var_param_col] = results['returns'].rolling(window).apply(
        lambda x: parametric_var_es_tstudent(x, alpha)[0], raw=False)
    results[es_param_col] = results['returns'].rolling(window).apply(
        lambda x: parametric_var_es_tstudent(x, alpha)[1], raw=False)

        #VaR histórico (percentil)
    results[var_hist_col] = results['returns'].rolling(window).quantile(1 - alpha)

        #ES histórico (percentil)
    results[es_hist_col] = results['returns'].rolling(window).apply(
        lambda x: historical_es(x, alpha), raw=False)

In [ ]:
window_data = results['returns'].iloc[-252:]
var95, es95 = parametric_var_es_tstudent(window_data, 0.95, nu=5)
var99, es99 = parametric_var_es_tstudent(window_data, 0.99, nu=5)

print(f"VaR 95% (t, ν=5): {var95:.4f}")
print(f"ES  95% (t, ν=5): {es95:.4f}")
print(f"VaR 99% (t, ν=5): {var99:.4f}")
print(f"ES  99% (t, ν=5): {es99:.4f}")

In [ ]:
# ------------------------------------------
# 5. VISUALIZACIÓN EN LA GRÁFICA
# ------------------------------------------

plt.figure(figsize=(32, 16))
plt.plot(results['returns'], label='Returns', color='black', alpha=0.4)

#Colores por tipo y nivel de confianza
colors = {
    'VaR_param_95': '#d62728',    # rojo oscuro
    'ES_param_95': '#ff9896',     # rojo claro
    'VaR_hist_95': '#1f77b4',     # azul oscuro
    'ES_hist_95': '#aec7e8',      # azul claro
    'VaR_param_99': '#2ca02c',    # verde oscuro
    'ES_param_99': '#98df8a',     # verde claro
    'VaR_hist_99': '#9467bd',     # morado oscuro
    'ES_hist_99': '#c5b0d5'       # morado claro
}

#Dibujamos líneas de VaR y ES para cada nivel de confianza
for alpha in alphas:
    a = int(alpha * 100)
    plt.plot(results[f'VaR_param_{a}'], label=f'VaR Paramétrico {a}%', linestyle='--', color=colors[f'VaR_param_{a}'])
    plt.plot(results[f'ES_param_{a}'], label=f'ES Paramétrico {a}%', linestyle='-.', color=colors[f'ES_param_{a}'])
    plt.plot(results[f'VaR_hist_{a}'], label=f'VaR Histórico {a}%', linestyle=':', color=colors[f'VaR_hist_{a}'])
    plt.plot(results[f'ES_hist_{a}'], label=f'ES Histórico {a}%', linestyle='-', linewidth=1.2, color=colors[f'ES_hist_{a}'])

plt.title(f'Rolling VaR & ES ({ticker}) de 252 días', fontsize=20)
plt.xlabel('Fecha', fontsize=16)
plt.ylabel('Retornos', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

(e) (20 puntos) Finalmente, para saber la eficiencia de nuestras estimaciones calcularemos el n ́umero de
veces que la p ́erdida fue superior a la estimaci ́on usando Var o ES, i.e. Definiremos violaci ́on si

$$r_{253} < VaR^t_α \quad o \quad ES^t_α$$

donde el $VaR^t_α \quad o \quad ES^t_α$ fue calculado con los retornos $r_1, r_2, ..., r_{252}$, o si

$$r_{254} < VaR^t_α \quad o \quad ES^t_α$$

donde el $VaR^t_α \quad o \quad ES^t_α$ fue calculado con los retornos $r_2, r_3, ..., r_{253}$, etc. En una tabla reporta el n ́umero
de violaciones y como porcentaje del tama ̃no de la muestra para cada nivel de confianza y cada medida
de riesgo. Tu c ́odigo debe generar esos resultados por si solo. Nota: Una buena estimaci ́on genera
un porcentaje de violaciones menores al $2.5 \%$.

In [ ]:
sum_r=[]

for col in results: #iteramos sobre cada encabezado
  #print(col)
  sum = 0
  if col != 'returns': #solo lo hacemos para los VaR y ES
    for i in range(len(results) - 1): #iteramos sobre todos los renglones
      if results[col][i]	!= np.nan: #iteramos sobre el rango de los no nulos
        if results['returns'][i+1] < results[col][i] : #comparamos si estimó bien o no
          sum += 1
    sum_r.append(sum)


sum_r=np.array(sum_r)#convertimos a arreglo de numpy

porc = (sum_r/len(results))*100 #calculamos el porcentaje de aciertos

estim = ['Buena estimación' if i < 2.5 else 'Mala estimación' for i in porc]

#Ahora creamos un dataframe que muestre el porcentaje y numero de fallos por cada VaR y ES
fallas = pd.DataFrame(
    {
        'VaR_param_95': [porc[0], sum_r[0], estim[0]],
        'ES_param_95': [porc[1], sum_r[1], estim[1]],
        'VaR_hist_95': [porc[2], sum_r[2], estim[2]],
        'ES_hist_95': [porc[3], sum_r[3], estim[3]],
        'VaR_param_99': [porc[4], sum_r[4], estim[4]],
        'ES_param_99': [porc[5], sum_r[5], estim[5]],
        'VaR_hist_99': [porc[6], sum_r[6], estim[6]],
        'ES_hist_99': [porc[7], sum_r[7], estim[7]],
    }
    , index=['Porcentaje de fallas','Número de fallas','Tipo de estimación']
)

In [ ]:
fallas